# **TabPFN Relational Benchmark**
This notebook benchmarks the performance of TabPFN models on datasets from RelBench in two scenarios:
1. **Single Table** – Using only the target entity table.
2. **Merged Table** – Using a naively denormalized table obtained by joining related tables.

It automates dataset loading, preprocessing (including date feature engineering), vectorization, model training, prediction, and evaluation for all compatible tasks within a chosen RelBench dataset. The results allow comparing model performance between single-table and merged-table configurations.


## Import Libraries

In [17]:
# --- Standard Library ---
import os
import time
import inspect
from typing import Any, Dict, List, Optional

# --- Third-Party Libraries ---
import numpy as np
import pandas as pd

# --- PyTorch / TorchFrame (already used) ---
import torch
from torch import Tensor
from torch_frame.config.text_embedder import TextEmbedderConfig

# --- Skrub / Sentence Transformers (keep as-is for your vectorization) ---
from skrub import TableVectorizer
from sentence_transformers import SentenceTransformer

# --- RelBench (keep as-is) ---
from relbench.datasets import get_dataset
from relbench.tasks import get_task, get_task_names
from relbench.base import TaskType
import relbench.metrics
from relbench.modeling.utils import get_stype_proposal
from relbench.modeling.graph import make_pkey_fkey_graph

# --- TabPFN (keep as-is) ---
from tabpfn import TabPFNClassifier, TabPFNRegressor

# --- getML (new) ---
from getml import data as gdata
from getml import engine as geng
from getml import pipeline as gpipeline
from getml import feature_learning as gfl
from getml import preprocessors as gprep
from getml import predictors as gpred

# --- Featuretools / Woodwork ---
import featuretools as ft
import woodwork as ww
from woodwork.logical_types import Categorical, IntegerNullable, Integer, NaturalLanguage, Double, Datetime

### Set Global Configuration

In [18]:
# Device selection (CPU, CUDA, or MPS if available)
def get_device():
    # Uncomment for auto-detection
    # if torch.backends.mps.is_available():
    #     return "mps"
    # elif torch.cuda.is_available():
    #     return "cuda"
    # else:
    #     return "cpu"
    return "cpu"  # Default: CPU

DEVICE = get_device()
print(f"Using device: {DEVICE}")

# Dataset and experiment settings
DATASET        = globals().get("DATASET", "rel-f1")           # Default dataset
SEED           = globals().get("SEED", 42)                    # Random seed
N_ESTIMATORS   = globals().get("N_ESTIMATORS", 16)            # TabPFN estimators
TABPFN_MAX     = globals().get("TABPFN_MAX", 1000)            # Max TabPFN samples
GETML_PROJECT  = globals().get("GETML_PROJECT", "default_project")  # getML project name

# getML engine state
_ENGINE_STARTED = False

# Optional: quiet Featuretools logs
ft.config.log_print_threshold = 1000000

Using device: cpu


### Featuretools knobs (safe defaults)

In [19]:
# ---- Featuretools knobs (safe defaults) ----
FT_MAX_DEPTH = globals().get("FT_MAX_DEPTH", 1)   # depth 1 keeps features compact & fast

# Desired primitives by intent (will be resolved to what's installed)
FT_AGG_PRIMITIVES_WISHLIST = [
    "mean", "sum", "count", "n_unique", "max", "min", "std", "mode"
]
FT_TRANS_PRIMITIVES_WISHLIST = [
    "day", "month", "year", "weekday", "hour"
]

# When building model inputs, drop raw join keys to avoid leakage / trivial joins
DROP_JOIN_KEYS_FROM_X = globals().get("DROP_JOIN_KEYS_FROM_X", True)


## Notebook Configuration and Dataset Selection


Sets the dataset name (`DATASET`) and download flag (`DOWNLOAD`), then discovers all available tasks for the selected dataset using RelBench’s APIs. Filters tasks to only those compatible with TabPFN (classification and regression).


In [20]:
# Reuse existing config if present, otherwise set defaults
DATASET = globals().get("DATASET", "rel-f1")
DOWNLOAD = globals().get("DOWNLOAD", True)

# Discover tasks and keep only entity-level cls/reg tasks TabPFN can handle
def _is_tabpfn_friendly(task):
    return task.task_type in (
        TaskType.BINARY_CLASSIFICATION,
        TaskType.MULTICLASS_CLASSIFICATION,
        TaskType.MULTILABEL_CLASSIFICATION,
        TaskType.REGRESSION,
    )

_all = get_task_names(DATASET)  # shown in tutorials
TASKS = []
for tname in _all:
    try:
        t = get_task(DATASET, tname, download=DOWNLOAD)
        if _is_tabpfn_friendly(t):
            TASKS.append(tname)
    except Exception as e:
        print(f"[skip] {tname}: {e!s}")

print(f"{DATASET}: {len(TASKS)} TabPFN-friendly tasks -> {TASKS}")


rel-f1: 3 TabPFN-friendly tasks -> ['driver-position', 'driver-dnf', 'driver-top3']


### Patch RelBench Metrics (Optional)

In [21]:
# Patch relbench.metrics.skm.mean_squared_error to local mean_squared_error
from sklearn.metrics import mean_squared_error

relbench.metrics.skm.mean_squared_error = mean_squared_error

def patched_rmse(true, pred):
    if "squared" in inspect.signature(mean_squared_error).parameters:
        return mean_squared_error(true, pred, squared=False)
    else:
        return np.sqrt(mean_squared_error(true, pred))

relbench.metrics.rmse = patched_rmse

### Text Embedding Configuration

This part provides a callable class to embed text columns using GloVe embeddings.

In [22]:
# Define a text embedding class using GloVe embeddings from Sentence Transformers.
class GloveTextEmbedding:
    def __init__(self, device: Optional[torch.device] = None):
        self.model = SentenceTransformer(
            "sentence-transformers/average_word_embeddings_glove.6B.300d",
            device=device,
        )

    def __call__(self, sentences: List[str]) -> Tensor:
        return torch.from_numpy(self.model.encode(sentences))

### Fetch Dataset Splits

Utility functions to load a task’s splits (`train`, `val`, `test`), convert them to pandas DataFrames, and extract features (`X`) and targets (`y`). Includes functions to:
* Load train/val/test splits for a task.
* Extract features/targets.
* Infer primary keys.
* Denormalize tables (one-hop join).
* Build data frames for both single-table and merged-table scenarios.


### Dataset loaders / frame builders

In [23]:
def fetch_splits(dataset_name: str, task_name: str, download: bool = True):
    task = get_task(dataset_name, task_name, download=download)
    # keep original columns (mask_input_cols=False so we see raw fields)
    splits = {
        split: task.get_table(split, mask_input_cols=False)
        for split in ("train", "val", "test")
    }
    return task, splits

def to_Xy(df: pd.DataFrame, target_col: str):
    y = df[target_col].to_numpy()
    X = df.drop(columns=[target_col])
    return X, y

def build_single_table_frames(task, splits):
    """
    Single-table mode: do NOT engineer features here.
    Just return raw base table X, y per split (target dropped from X).
    """
    frames = {}
    for split, table in splits.items():
        df = table.df.copy()
        X, y = to_Xy(df, task.target_col)
        frames[split] = (X, y, df)
    return frames


### Featuretools helpers: dtype cleanup, PK/FK inference, ES builders

In [24]:
def _clean_for_ft(df: pd.DataFrame) -> pd.DataFrame:
    """
    Make pandas dtypes friendly to Woodwork/Featuretools:
    - Keep nullable ints as IntegerNullable where possible,
    - Avoid mixed/object weirdness where a simple cast works.
    """
    x = df.copy()
    # Avoid pandas NA strings in key-like columns
    for c in x.columns:
        if pd.api.types.is_object_dtype(x[c]) and x[c].dtype != "string":
            # do nothing: TableVectorizer handles object; FT will type via Woodwork
            pass
        # Ensure datetimes are parsed
        if not pd.api.types.is_datetime64_any_dtype(x[c]):
            # try-to-datetime only if it looks like dates by name
            low = str(c).lower()
            if any(k in low for k in ("date", "time", "ts", "timestamp")):
                try:
                    x[c] = pd.to_datetime(x[c], errors="ignore")
                except Exception:
                    pass
    return x

def _candidate_pk(df: pd.DataFrame, table_name: str) -> Optional[str]:
    """
    Deterministic PK selection:
      1) exact 'id' or f'{table}_id' or any '*_id' that is unique and non-null
      2) otherwise any column that is fully unique and non-null
      3) else None  (we will make an index)
    """
    cols = list(df.columns)
    priorities = [ "id", f"{table_name}_id" ] + [c for c in cols if c.lower().endswith("_id")]
    def is_good_key(s: pd.Series) -> bool:
        return s.notna().all() and s.is_unique
    for c in priorities:
        if c in df.columns and is_good_key(df[c]):
            return c
    for c in df.columns:
        s = df[c]
        # treat nullable-int as fine if no NA
        if is_good_key(s):
            return c
    return None

def _infer_pk_fk_graph(tables: Dict[str, pd.DataFrame]) -> Dict[str, Dict[str, List[Dict[str, str]]]]:
    """
    Returns a schema dict with:
      { "pkeys": {table: pk_col_or_None}, "fkeys": {parent: [ {child, fk}, ... ] } }

    FK detection rule: a child column qualifies as FK to parent PK if:
      - dtypes are compatible (string/int-ish), and
      - coverage >= 0.98: child[~na] values appear in parent PK values w/ that rate
    """
    pkeys: Dict[str, Optional[str]] = {}
    for tname, df in tables.items():
        pkeys[tname] = _candidate_pk(df, tname)

    # Build lookup for parent unique sets
    parent_values = {}
    for parent, pk in pkeys.items():
        if pk is None:
            continue
        vals = pd.Series(tables[parent][pk]).dropna().unique()
        parent_values[parent] = set(vals.tolist())

    fkeys: Dict[str, List[Dict[str, str]]] = {parent: [] for parent in tables.keys()}
    for parent, pk in pkeys.items():
        if pk is None:
            continue
        pset = parent_values.get(parent, set())
        if not pset:
            continue
        for child, cdf in tables.items():
            if child == parent:
                continue
            # cheap prefilter of compatible columns
            for col in cdf.columns:
                if col == pk or col.lower() == pk.lower() or col.lower().endswith("_id"):
                    s = cdf[col].dropna()
                    if len(s) == 0:
                        continue
                    # try to compare as strings to be robust to dtype differences
                    covered = (s.astype(str).isin(pd.Series(list(pset)).astype(str))).mean()
                    if covered >= 0.98:
                        fkeys[parent].append({"child": child, "fk": col})
    return {"pkeys": pkeys, "fkeys": fkeys}

def _make_es_for_split(base_name: str,
                       pop_df: pd.DataFrame,
                       all_tables: Dict[str, pd.DataFrame],
                       schema: Dict[str, Dict[str, Any]]) -> ft.EntitySet:
    """
    Build a Featuretools EntitySet for the given split's population table,
    reusing the same PK/FK schema.
    """
    es = ft.EntitySet(id=f"rb_es_{base_name}")
    # Add population first
    pop_df = _clean_for_ft(pop_df)
    pop_pk = schema["pkeys"].get(base_name)
    if pop_pk is None or pop_pk not in pop_df.columns:
        es = es.add_dataframe(dataframe_name=base_name, dataframe=pop_df,
                              index=f"{base_name}__ft_index", make_index=True)
        pop_pk = f"{base_name}__ft_index"
    else:
        es = es.add_dataframe(dataframe_name=base_name, dataframe=pop_df, index=pop_pk)

    # Add peripherals
    for tname, df in all_tables.items():
        if tname == base_name:
            continue
        df2 = _clean_for_ft(df)
        pk = schema["pkeys"].get(tname)
        if pk is None or pk not in df2.columns:
            es = es.add_dataframe(dataframe_name=tname, dataframe=df2,
                                  index=f"{tname}__ft_index", make_index=True)
        else:
            es = es.add_dataframe(dataframe_name=tname, dataframe=df2, index=pk)

    # Add relationships
    for parent, rels in schema["fkeys"].items():
        # skip children if parent not present (unlikely)
        if parent not in es.dataframe_dict:
            continue
        parent_pk = schema["pkeys"].get(parent) or f"{parent}__ft_index"
        for r in rels:
            child = r["child"]
            fk = r["fk"]
            if child not in es.dataframe_dict:
                continue
            # If FK missing in child (after cleanup), skip
            if fk not in es[child].ww.columns:
                continue
            es = es.add_relationship(parent_dataframe_name=parent,
                                     parent_column_name=parent_pk,
                                     child_dataframe_name=child,
                                     child_column_name=fk)
    return es

def _dfs_feature_matrices(es_train: ft.EntitySet,
                          es_val: ft.EntitySet,
                          es_test: ft.EntitySet,
                          target_df: str):
    """
    Run DFS on train, then calculate the same features on val/test for perfect alignment.
    Resolves primitives against the locally installed Featuretools.
    """
    agg_prims, trans_prims = _resolve_ft_primitives(
        FT_AGG_PRIMITIVES_WISHLIST, FT_TRANS_PRIMITIVES_WISHLIST
    )

    fm_train, fdefs = ft.dfs(
        entityset=es_train,
        target_dataframe_name=target_df,
        agg_primitives=agg_prims,
        trans_primitives=trans_prims,
        max_depth=FT_MAX_DEPTH,
        features_only=False,
        verbose=False
    )
    fm_val = ft.calculate_feature_matrix(features=fdefs, entityset=es_val, verbose=False)
    fm_test = ft.calculate_feature_matrix(features=fdefs, entityset=es_test, verbose=False)

    # Align column order across splits
    fm_val = fm_val.reindex(columns=fm_train.columns, fill_value=np.nan)
    fm_test = fm_test.reindex(columns=fm_train.columns, fill_value=np.nan)
    return fm_train, fm_val, fm_test, fdefs

### Resolve FT primitive names to what's actually available

In [25]:
def _resolve_ft_primitives(
    agg_wishlist: list,
    trans_wishlist: list
):
    """
    Inspects installed Featuretools primitives and returns (agg_list, trans_list)
    with only supported names, mapping common aliases where possible.
    """
    prim_df = ft.primitives.list_primitives()
    have_agg = set(prim_df[prim_df.type == "aggregation"].name.str.lower())
    have_trans = set(prim_df[prim_df.type == "transform"].name.str.lower())

    # Common alias sets for version differences
    alias_groups_agg = [
        {"n_unique", "nunique", "num_unique", "count_unique"},
        {"mode", "mode_agg"},  # sometimes exposed with a suffix
        {"std", "standard_deviation"},
        {"count"}  # count is stable but keep set logic uniform
    ]
    alias_groups_trans = [
        {"weekday", "week_day"},
        {"year"},
        {"month"},
        {"day"},
        {"hour"},
    ]

    def pick_available(wishlist, have_set, alias_groups):
        out = []
        lower_wish = [w.lower() for w in wishlist]
        for w in lower_wish:
            # direct
            if w in have_set:
                out.append(w)
                continue
            # alias
            matched = False
            for group in alias_groups:
                if w in group:
                    # pick first available from group
                    for cand in group:
                        if cand in have_set:
                            out.append(cand)
                            matched = True
                            break
                if matched:
                    break
            # if still not matched, skip silently
        # Deduplicate while preserving order
        seen = set()
        uniq = []
        for p in out:
            if p not in seen:
                seen.add(p)
                uniq.append(p)
        return uniq

    agg = pick_available(agg_wishlist, have_agg, alias_groups_agg)
    trans = pick_available(trans_wishlist, have_trans, alias_groups_trans)

    if len(agg) == 0:
        # minimally guarantee some aggregations
        for fallback in ["mean", "sum", "count", "max", "min"]:
            if fallback in have_agg:
                agg.append(fallback)
        agg = list(dict.fromkeys(agg))  # dedupe

    return agg, trans


### getML Engine Helpers

In [26]:
def _ensure_getml_engine():
    """Idempotently launch getML engine and set project."""
    global _ENGINE_STARTED, GETML_PROJECT
    if _ENGINE_STARTED:
        return
    try:
        geng.launch(
            allow_push_notifications=False,
            allow_remote_ips=False,
            in_memory=True,
            launch_browser=False,
            log=False,
            quiet=True,
        )
        geng.set_project(GETML_PROJECT)
        _ENGINE_STARTED = True
    except Exception as e:
        raise RuntimeError("Failed to launch the getML engine.") from e

def _shutdown_getml_engine():
    """Shutdown getML engine if started."""
    global _ENGINE_STARTED
    if not _ENGINE_STARTED:
        return
    try:
        geng.shutdown()
    except Exception:
        pass
    _ENGINE_STARTED = False

def _first_datetime_col(df: pd.DataFrame) -> Optional[str]:
    # Prefer already-typed datetime64 columns
    for col in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            return col
    # Try to parse common timestamp strings without mutating df
    for col in df.columns:
        if df[col].dtype == object:
            try:
                pd.to_datetime(df[col], errors="ignore")
                return col
            except Exception:
                pass
    return None


def _guess_primary_key(table_name: str, df: pd.DataFrame) -> Optional[str]:
    candidates = [
        f"{table_name}_id",
        f"{table_name[:-1]}_id" if table_name.endswith("s") else None,
        "id",
        "ID",
        f"{table_name}Id",
        f"{table_name}_pk",
    ]
    candidates = [c for c in candidates if c and c in df.columns]
    # prefer unique columns
    for c in candidates:
        if df[c].is_unique:
            return c
    return candidates[0] if candidates else None


def _guess_foreign_key(pop_df: pd.DataFrame, per_name: str, per_pk: str) -> Optional[str]:
    if per_pk in pop_df.columns:
        return per_pk
    candidates = [
        f"{per_name}_id",
        f"{per_name[:-1]}_id" if per_name.endswith("s") else None,
        f"{per_name}{per_pk.capitalize()}" if per_pk != "id" else None,
    ]
    for c in [c for c in candidates if c]:
        if c in pop_df.columns:
            return c
    return None


def _roles_from_df(
    df: pd.DataFrame,
    target_col: Optional[str],
    join_keys: List[str],
    time_col: Optional[str],
):
    num_cols, cat_cols = [], []
    skip = set(join_keys)
    if time_col:
        skip.add(time_col)
    if target_col:
        skip.add(target_col)

    for c in df.columns:
        if c in skip:
            continue
        if pd.api.types.is_numeric_dtype(df[c]):
            num_cols.append(c)
        else:
            cat_cols.append(c)

    return {
        "target": target_col,
        "join_key": join_keys,
        "time_stamp": time_col,
        "numerical": num_cols,
        "categorical": cat_cols,
    }


def _df_set_roles(gdf: gdata.DataFrame, roles: Dict[str, Any]):
    # Assign roles on a getML DataFrame
    if roles.get("target"):
        gdf.set_role(roles["target"], gdata.roles.target)
    jk = roles.get("join_key") or []
    if jk:
        gdf.set_role(jk, gdata.roles.join_key)
    if roles.get("time_stamp"):
        gdf.set_role(roles["time_stamp"], gdata.roles.time_stamp)
    if roles.get("numerical"):
        gdf.set_role(roles["numerical"], gdata.roles.numerical)
    if roles.get("categorical"):
        gdf.set_role(roles["categorical"], gdata.roles.categorical)


def _harmonize_join_keys(pop_by_split: Dict[str, pd.DataFrame], per_spec: List[Dict[str, str]]) -> List[Dict[str, str]]:
    """
    Ensure the population has columns named like each peripheral's PK.
    If population FK name != peripheral PK name, mirror FK into a new column
    with the PK name (required because getML matches join_keys by name).
    Returns the filtered per_spec with only peripherals we can actually join.
    """
    valid_specs = []
    for spec in per_spec:
        pk = spec["pk"]
        fk = spec["fk"]
        can_use = True
        for split, df in pop_by_split.items():
            if pk in df.columns:
                continue  # already aligned
            if fk in df.columns:
                df[pk] = df[fk]
            else:
                can_use = False
                break
        if can_use:
            valid_specs.append(spec)
    return valid_specs


### Build Merged-Table Frames

In [27]:
def build_merged_table_frames(dataset, task, splits):
    """
    Build merged features with Featuretools:
    - Detect PK/FK pairs from the raw tables (deterministic rules, no guessing),
    - Build an EntitySet per split with the same relationships,
    - Generate relational aggregates via DFS,
    - Drop raw join keys (optional) and target from inputs,
    - Return (X, y, engineered_df) per split to match the rest of the notebook.
    """
    # --- Load DB tables (pandas) ---
    db = dataset.get_db()
    all_tables: Dict[str, pd.DataFrame] = {
        name: (tbl.df.copy() if hasattr(tbl, "df") else tbl.to_pandas())
        for name, tbl in db.table_dict.items()
    }

    # Identify the population table name used by RelBench task tables
    base_name = getattr(splits["train"], "name", "population")

    # Clean tables for FT typing
    all_tables = {k: _clean_for_ft(v) for k, v in all_tables.items()}

    # --- Infer PK/FK graph using Woodwork typing + coverage checks ---
    schema = _infer_pk_fk_graph(all_tables)

    # If FT didn't find a PK for the population, ensure we produce one later
    if schema["pkeys"].get(base_name) is None and base_name in all_tables:
        # let _make_es_for_split create an artificial index
        pass

    # --- Build EntitySets per split ---
    pop = {split: tbl.df.copy() for split, tbl in splits.items()}  # keep your original behavior
    es_train = _make_es_for_split(base_name, pop["train"], all_tables, schema)
    es_val   = _make_es_for_split(base_name, pop["val"],   all_tables, schema)
    es_test  = _make_es_for_split(base_name, pop["test"],  all_tables, schema)

    # --- DFS -> aligned feature matrices ---
    fe_train, fe_val, fe_test, feature_defs = _dfs_feature_matrices(es_train, es_val, es_test, target_df=base_name)

    # --- Prepare (X, y), optionally drop raw join keys from model inputs ---
    def _drop_keys(dfX: pd.DataFrame) -> pd.DataFrame:
        if not DROP_JOIN_KEYS_FROM_X:
            return dfX
        drop_cols = set()

        # population PK
        pop_pk = schema["pkeys"].get(base_name)
        if pop_pk and pop_pk in dfX.columns:
            drop_cols.add(pop_pk)

        # any FKs that appear on population table columns
        for parent, rels in schema["fkeys"].items():
            for r in rels:
                fk = r["fk"]
                if fk in dfX.columns:
                    drop_cols.add(fk)

        return dfX.drop(columns=list(drop_cols), errors="ignore")

    def _to_Xy_safe(fe_df: pd.DataFrame, raw_df: pd.DataFrame):
        # Ensure target is not part of X
        X = fe_df.copy()
        if task.target_col in X.columns:
            X = X.drop(columns=[task.target_col], errors="ignore")
        X = _drop_keys(X)
        # y from the raw population split
        y = raw_df[task.target_col].to_numpy()
        return X, y

    Xtr, ytr = _to_Xy_safe(fe_train, pop["train"])
    Xva, yva = _to_Xy_safe(fe_val,   pop["val"])
    Xte, yte = _to_Xy_safe(fe_test,  pop["test"])

    return {
        "train": (Xtr, ytr, fe_train),
        "val":   (Xva, yva, fe_val),
        "test":  (Xte, yte, fe_test),
    }


## Vectorization Wrapper (Version-Safe)

Initializes a `TableVectorizer` with only supported arguments for the installed `skrub` or `dirty_cat` version, ensuring compatibility. Transforms `train`, `val`, and `test` splits into numerical feature matrices, converting them to dense format if necessary.


### Helper Functions for Vectorization

In [28]:
# This function creates a `TableVectorizer` instance with version-specific arguments.
def _make_table_vectorizer():
    sig = inspect.signature(TableVectorizer.__init__)
    allowed = set(sig.parameters.keys()) - {"self"}

    tv_kwargs = {}

    # Only set kwargs that actually exist in the installed version
    if "cardinality_threshold" in allowed:
        tv_kwargs["cardinality_threshold"] = globals().get("CARDINALITY_THRESHOLD", 1000)

    # Some versions expose this; others don't, guard it
    if "high_cardinality_transformer" in allowed:
        tv_kwargs["high_cardinality_transformer"] = globals().get("HIGH_CARD_TRANSFORMER", "hashing")

    # Optional knobs if you define them globally and the version supports them
    if "text_separator" in allowed and "TEXT_SEPARATOR" in globals():
        tv_kwargs["text_separator"] = globals()["TEXT_SEPARATOR"]
    if "numerical_transformer" in allowed and "NUMERICAL_TRANSFORMER" in globals():
        tv_kwargs["numerical_transformer"] = globals()["NUMERICAL_TRANSFORMER"]
    if "categorical_transformer" in allowed and "CATEGORICAL_TRANSFORMER" in globals():
        tv_kwargs["categorical_transformer"] = globals()["CATEGORICAL_TRANSFORMER"]

    return TableVectorizer(**tv_kwargs)

# Converts a sparse matrix to a dense NumPy array, handling cases where the input is already dense or does not support `.toarray()`.
def _to_dense(X):
    try:
        # scipy sparse matrices have .toarray()
        return X.toarray() if hasattr(X, "toarray") else X
    except Exception:
        return X

# Vectorizes the training, validation, and test splits using a `TableVectorizer`. It initializes the vectorizer, fits it on the training data, and transforms all splits into dense NumPy arrays. Returns the vectorizer and the transformed matrices.
def vectorize_splits(X_train, X_val, X_test):
    # Fit only on training data to prevent data leakage
    tv = _make_table_vectorizer()
    Xt = _to_dense(tv.fit_transform(X_train))
    Xv = _to_dense(tv.transform(X_val))
    Xs = _to_dense(tv.transform(X_test))
    return tv, Xt, Xv, Xs


### Training and Prediction Helpers

In [29]:
# This function subsamples the training data to a maximum size defined by `TABPFN_MAX`. If the dataset is smaller than this cap, it returns the full dataset; otherwise, it randomly samples without replacement.
def _subsample(X, y, cap=TABPFN_MAX, seed=SEED):
    if len(X) <= cap:
        return X, y, np.arange(len(X))
    idx = np.random.RandomState(seed).choice(len(X), size=cap, replace=False)
    if hasattr(X, "iloc"):
        Xs = X.iloc[idx]
    else:
        Xs = X[idx]
    ys = y[idx]
    return Xs, ys, idx

# Fits a TabPFN model (either classifier or regressor) based on the task type. It initializes the model with the specified device and number of estimators, then fits it to the provided training data.
def _fit_tabpfn(task, Xt, yt):
    if task.task_type == TaskType.REGRESSION and TabPFNRegressor is not None:
        model = TabPFNRegressor(
            device=DEVICE,
            #n_estimators=int(N_ESTIMATORS),
            ignore_pretraining_limits=True,
        )
    else:
        model = TabPFNClassifier(
            device=DEVICE,
            #n_estimators=int(N_ESTIMATORS),
            ignore_pretraining_limits=True,
        )
    model.fit(Xt, yt)
    return model

# Helper function to make predictions for a given task using the fitted model. It handles different task types (regression, binary classification, multiclass/multilabel) and returns the appropriate prediction format.
def _predict_for_task(task, model, X):
    # align with RelBench evaluators: AUROC expects probabilities for the positive class
    if task.task_type == TaskType.REGRESSION:
        return model.predict(X)
    proba = model.predict_proba(X)
    if task.task_type == TaskType.BINARY_CLASSIFICATION:
        return proba[:, 1]
    else:
        # multiclass/multilabel: pass full probability matrix
        return proba

### Run TabPFN on Selected Tasks

Runs TabPFN on a specified dataset and task, handling both single-table and merged-table modes. It vectorizes the data, fits the model, makes predictions, and evaluates performance using RelBench’s evaluators. Returns a dictionary with results.

In [30]:
def run_tabpfn_on_task(dataset_name: str, task_name: str, mode: str = "single") -> Dict[str, Any]:
    # Load dataset and task splits
    dataset = get_dataset(dataset_name, download=DOWNLOAD)
    task, splits = fetch_splits(dataset_name, task_name, download=DOWNLOAD)

    # Ensure the task is compatible with TabPFN
    if mode == "single":
        frames = build_single_table_frames(task, splits)
    elif mode == "merged":
        try:
            frames = build_merged_table_frames(dataset, task, splits)
        except Exception as e:
            print(f"merged mode failed: {e}, falling back to single-table mode.")
            raise
    else:
        raise ValueError("mode must be 'single' or 'merged'")

    # Extract features and targets for each split
    (Xtr, ytr, _dftr) = frames["train"]
    (Xva, yva, dfva)  = frames["val"]
    (Xte, yte, dfte)  = frames["test"]

    # Vectorize
    tv, Xt, Xv, Xs = vectorize_splits(Xtr, Xva, Xte)

    # Respect TabPFN's sample cap
    Xt_cap, yt_cap, _ = _subsample(Xt, ytr, cap=TABPFN_MAX, seed=SEED)

    # Fit
    model = _fit_tabpfn(task, Xt_cap, yt_cap)

    # Predict & Evaluate with RelBench evaluators
    val_pred  = _predict_for_task(task, model, Xv)
    test_pred = _predict_for_task(task, model, Xs)

    # Align predictions with original DataFrame indices for evaluation
    val_metrics  = task.evaluate(val_pred,  splits["val"])
    test_metrics = task.evaluate(test_pred, splits["test"])

    # Convert metrics to a dictionary, ensuring all values are floats
    out = {
        "dataset": dataset_name,
        "task": task_name,
        "mode": mode,
        "val_metrics": val_metrics,
        "test_metrics": test_metrics,
        "n_train_used": len(Xt_cap),
        "n_train_total": len(Xt),
        "n_val": len(Xv),
        "n_test": len(Xs),
    }
    return out

## Orchestrator for Benchmark Runs

Iterates over all discovered tasks and runs TabPFN in both **single** and **merged** modes. Collects performance metrics for validation and test splits into a results table, handling failures gracefully. Sorts results for easier comparison.


In [31]:
MODES = globals().get("MODES", ["single", "merged"])

records = []
failures = []

# Run TabPFN on all tasks in both modes and collect results
for task_name in TASKS:
    for mode in MODES:
        try:
            res = run_tabpfn_on_task(DATASET, task_name, mode=mode)
            # Flatten metrics for val and test, one metric per row
            for split in ["val", "test"]:
                metrics = res.get(f"{split}_metrics") or {}
                for metric_name, metric_value in metrics.items():
                    # Only add rows with non-empty metric_value
                    if metric_value is not None and not (isinstance(metric_value, float) and np.isnan(metric_value)):
                        records.append({
                            "dataset": res.get("dataset", DATASET),
                            "task": res.get("task", task_name),
                            "split": split,
                            "mode": res.get("mode", mode),
                            "method": "TabPFN_experimental_v1.0",
                            "metric": metric_name,
                            "score": metric_value,
                        })
        except Exception as e:
            msg = f"[{DATASET} | {task_name} | {mode}] failed: {e!s}"
            print(msg)
            failures.append(msg)

# Convert collected records into a DataFrame
results_df = pd.DataFrame.from_records(records)

# If no successful runs were recorded, display a message and create an empty DataFrame
if results_df.empty:
    print("No successful runs were recorded. Check the failure messages above.")
    results_df = pd.DataFrame(
        columns=["dataset", "task", "split", "mode", "method", "metric", "score"]
    )
else:
    # Ensure required sort keys exist even if some rows missed them
    for col in ["task", "mode"]:
        if col not in results_df.columns:
            results_df[col] = pd.NA
    # Sort only by the columns that exist to avoid KeyError
    sort_keys = [c for c in ["task", "mode", "metric"] if c in results_df.columns]
    if sort_keys:
        results_df = results_df.sort_values(sort_keys)

display(results_df)


/Users/michaelflppv/PycharmProjects/TabPFN/.venv1/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/michaelflppv/PycharmProjects/TabPFN/.venv1/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/var/folders/yl/k2ptpp6s2ms3m1kvq65cr07m0000gn/T/ipykernel_91356/1664296673.py:19: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  x[c] = pd.to_datetime(x[c], errors="ignore")
/var/folders/yl/k2ptpp6s2ms3m1kvq65cr07m0000gn/T/ipykernel_91356/1664296673.py:19: FutureWarning: errors='igno

,dataset,task,split,mode,method,metric,score
21,rel-f1,driver-dnf,val,merged,TabPFN_experimental_v1.0,accuracy,0.779152
25,rel-f1,driver-dnf,test,merged,TabPFN_experimental_v1.0,accuracy,0.705128
20,rel-f1,driver-dnf,val,merged,TabPFN_experimental_v1.0,average_precision,0.781151
24,rel-f1,driver-dnf,test,merged,TabPFN_experimental_v1.0,average_precision,0.734583
22,rel-f1,driver-dnf,val,merged,TabPFN_experimental_v1.0,f1,0.875869
26,rel-f1,driver-dnf,test,merged,TabPFN_experimental_v1.0,f1,0.827068
23,rel-f1,driver-dnf,val,merged,TabPFN_experimental_v1.0,roc_auc,0.486295
27,rel-f1,driver-dnf,test,merged,TabPFN_experimental_v1.0,roc_auc,0.516074
13,rel-f1,driver-dnf,val,single,TabPFN_experimental_v1.0,accuracy,0.779152
17,rel-f1,driver-dnf,test,single,TabPFN_experimental_v1.0,accuracy,0.705128


### Save Results to CSV

In [32]:
# Specify output directory
out_dir = globals().get("OUT_DIR", "outputs")
os.makedirs(out_dir, exist_ok=True)

# Change timestamp format to "dd.mm.yyyy-hh:mm"
timestamp = time.strftime("%d.%m.%Y-%H:%M")
csv_name = f"tabpfn_{DATASET}_{timestamp}.csv"
csv_path = os.path.join(out_dir, csv_name)

# Round all numerical results to 4 decimal places before saving
if "score" in results_df.columns:
    if pd.api.types.is_numeric_dtype(results_df["score"]):
        results_df["score"] = results_df["score"].round(4)
    else:
        # Optionally, try to convert to numeric first
        results_df["score"] = pd.to_numeric(results_df["score"], errors="coerce").round(4)

# Filter out rows with empty score values before saving
if "score" in results_df.columns:
    results_df = results_df[results_df["score"].notnull()]

# Save the results DataFrame to a CSV file
results_df.to_csv(csv_path, index=False)
print(f"Saved results to: {csv_path}")


Saved results to: outputs/tabpfn_rel-f1_20.08.2025-22:07.csv
